In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

home_dir = '/users/cusgadmin'
# home_dir = '/Users/alvinyuan/Documents/Vision'

# Make sure that caffe is on the python path:
caffe_root = '%s/caffe/' % home_dir  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

# Set the right path to your model definition file, pretrained model weights,
# and the image you would like to classify.
MODEL_FILE = '%s/caffe/models/bvlc_reference_caffenet/deploy.prototxt' % home_dir
PRETRAINED = '%s/caffe/models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel' % home_dir

import os
if not os.path.isfile(PRETRAINED):
    print("Downloading pre-trained CaffeNet model...")
    !~/caffe/scripts/download_model_binary.py ~/caffe/models/bvlc_reference_caffenet

In [ ]:
caffe.set_mode_cpu()
mean_image = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
print mean_image.shape
net = caffe.Classifier(MODEL_FILE, PRETRAINED,
                       mean=mean_image.mean(1).mean(1),
                       channel_swap=(2,1,0),
                       raw_scale=255,
                       image_dims=(256, 256))

In [ ]:
IMAGE_FILE = 'images/kettle.jpg'
input_image = caffe.io.load_image(IMAGE_FILE)
plt.imshow(input_image)

In [ ]:
! $HOME/caffe/data/ilsvrc12/get_ilsvrc_aux.sh

In [ ]:
import pandas as pd
with open('%s/data/ilsvrc12/synset_words.txt' % caffe_root) as f:
    labels_df = pd.DataFrame([
        {
            'synset_id': l.strip().split(' ')[0],
            'name': ' '.join(l.strip().split(' ')[1:]).split(',')[0]
        }
        for l in f.readlines()
    ])
labels_df.sort('synset_id')
labels_dict = labels_df.set_index('synset_id').to_dict()['name'] # maps synset_id to name
print labels_dict

In [ ]:
prediction = net.predict([input_image])  # predict takes any number of images, and formats them for the Caffe net automatically
print 'prediction shape:', prediction[0].shape
plt.plot(prediction[0])
predicted_idx = prediction[0].argmax()
print 'predicted class:', predicted_idx
print labels_df['name'][predicted_idx]

In [ ]:
import os
import cv2

#
#   Run sementation.bounding_boxes.py prior to running this script
#
bbx_folder = '../region_proposals/'

file_list = os.listdir(bbx_folder)

image_list = [caffe.io.load_image(bbx_folder + x) for x in file_list]

for image in image_list:
    plt.imshow(image)
    plt.show()

In [ ]:
prediction = net.predict(image_list)  # predict takes any number of images, and formats them for the Caffe net automatically
for i in range(len(prediction)):
    predicted_idx   = prediction[i].argmax()
    predicted_class = labels_df['name'][predicted_idx]
    plt.subplot(121)
    plt.imshow(image_list[i])
    plt.subplot(122)
    plt.plot(prediction[i])
    plt.title(predicted_class)
    plt.show()
    